In [39]:
import os
import tensorflow_hub as hub
import numpy as np
from langchain_postgres import PGVector
from langchain_core.documents import Document

In [17]:
# Initialize Embedding Model
class EmbeddingModel:
    def __init__(self, model_url="https://tfhub.dev/google/universal-sentence-encoder/4"):
        """
        Initialize the embedding model with the given model URL.
        """
        print("Loading embedding model...")
        self.model = hub.load(model_url)
        print("Embedding model loaded successfully.")

    def embed_query(self, query, **kwargs):
        """
        Generate embedding for a single query (string).
        """
        return self.model([query]).numpy()[0]

    def embed_documents(self, documents):
        """
        Generate embeddings for a list of documents (strings).
        """
        return self.model(documents).numpy()

# Initialize the embedding model
embeddings = EmbeddingModel()
print("Embedding model initialization complete.")

Loading embedding model...
Embedding model loaded successfully.
Embedding model initialization complete.


In [18]:
# Define PostgreSQL Connection String
conn_string = "Paste your PostgreSQL connection string here"
collection_name = "example_documents"
print("Connection string and collection name defined.")

Connection string and collection name defined.


In [19]:
# Initialize PGVector vector store
vector_store = PGVector(
    embeddings=embeddings,
    collection_name=collection_name,
    connection=conn_string,
    use_jsonb=True,
)
print("PGVector vector store initialized.")

PGVector vector store initialized.


In [20]:
# Add Documents to Vector Store
docs = [
    Document(
        page_content="The quick brown fox jumps over the lazy dog.",
        metadata={"id": 1, "topic": "animals"},
    ),
    Document(
        page_content="Artificial intelligence is transforming the world.",
        metadata={"id": 2, "topic": "technology"},
    ),
    Document(
        page_content="Healthy eating is important for a balanced lifestyle.",
        metadata={"id": 3, "topic": "health"},
    ),
]
print("Documents defined for addition to vector store.")

Documents defined for addition to vector store.


In [21]:
# Add the documents to the vector store
vector_store.add_documents(docs, ids=[doc.metadata["id"] for doc in docs])
print("Documents added successfully to vector store.")


Documents added successfully to vector store.


In [24]:
# Query the Vector Store
query_text = "Tell me about AI."
query_embedding = embeddings.embed_query(query_text)
print("Query embedding generated.")

Query embedding generated.


In [30]:
results = vector_store.similarity_search_with_score(query=query_text, k=3)
print("\n--- Query Results ---")
results = results[0][0].page_content
print("Similarity search complete.")
print(results)



--- Query Results ---
Similarity search complete.
Artificial intelligence is transforming the world.


In [36]:
# Transform vector store into a retriever
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
print("Retriever created from vector store.")



Retriever created from vector store.


In [37]:
retriever_results = retriever.invoke(input=query_text)
print("Retriever invoked for query.")

Retriever invoked for query.


In [38]:
print("\n--- Retriever Results ---")
for doc in retriever_results:
    print(f"* {doc.page_content} [{doc.metadata}]")
print("Notebook execution complete.")


--- Retriever Results ---
* Artificial intelligence is transforming the world. [{'id': 2, 'topic': 'technology'}]
Notebook execution complete.
